 This project will explore house prices in major cities around ontario

- Import packages and read in csvs scraped using Octoparse
- Append all dataframes into one dataframe

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

In [4]:
osh = pd.read_csv('Oshawahomes.csv')
oak = pd.read_csv('Oakhomes.csv')
markh = pd.read_csv('Markhamhomes.csv')
whit = pd.read_csv('Whitbyhomes.csv')
rich = pd.read_csv('Richmondhomes.csv')
newm = pd.read_csv('Newmhomes.csv')
ajaxh = pd.read_csv('Ajaxhomes.csv')
milt = pd.read_csv('Miltonhomes.csv')
vaug = pd.read_csv('Vaughanhomes.csv')
brap = pd.read_csv('Bramptonhomes.csv')
burl = pd.read_csv('Burlingtonhomes.csv')
missi = pd.read_csv('Missihomes.csv')

In [49]:
#append the dataframes to form 1 record 
houses = pd.DataFrame().append([osh,oak, markh,whit,rich,newm,ajaxh,milt,vaug,brap,burl,missi])
houses.shape

(53632, 10)

In [50]:
houses.head()

,Title,Title_URL,Image,Tag,Price,Price1,style_component__qn84q,style_listingdetail__b0bdg,style_listingdetail__b0bdg2,style_listingdetail__b0bdg3
0,362 Newgate Ave,https://www.zoocasa.com/oshawa-on-real-estate/...,https://www.zoocasa.com/_next/image?url=https%...,Sold,"$930,000","List Price$950,000",2 days,3 Bed,2 Bath,1500–2000 sqft
1,1488 Fenelon Cres,https://www.zoocasa.com/oshawa-on-real-estate/...,https://www.zoocasa.com/_next/image?url=https%...,Sold,"$730,000","List Price$699,000",3 days,3 Bed,2 Bath,N/A sqft
2,536 Park Rd S,https://www.zoocasa.com/oshawa-on-real-estate/...,https://www.zoocasa.com/_next/image?url=https%...,Sold,"$715,000","List Price$759,300",2 days,3 Bed,3 Bath,N/A sqft
3,292 Elgin St W,https://www.zoocasa.com/oshawa-on-real-estate/...,https://www.zoocasa.com/_next/image?url=https%...,Sold,"$585,880","List Price$599,000",Sold 2 Days Ago,2 Bed,2 Bath,N/A sqft
4,148 Colborne St E,https://www.zoocasa.com/oshawa-on-real-estate/...,https://www.zoocasa.com/_next/image?url=https%...,Sold,"$698,000","List Price$658,000",3 days,3 Bed,1 Bath,N/A sqft


Checking the datatype for the columns
some columns need to be renamed and cleaned 

In [8]:
#check data types for different columns 
houses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53632 entries, 0 to 10268
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Title                        53632 non-null  object
 1   Title_URL                    53632 non-null  object
 2   Image                        53623 non-null  object
 3   Tag                          53632 non-null  object
 4   Price                        53632 non-null  object
 5   Price1                       53632 non-null  object
 6   style_component__qn84q       53632 non-null  object
 7   style_listingdetail__b0bdg   53632 non-null  object
 8   style_listingdetail__b0bdg2  53632 non-null  object
 9   style_listingdetail__b0bdg3  53632 non-null  object
dtypes: object(10)
memory usage: 4.5+ MB


In [9]:
houses.columns

Index(['Title', 'Title_URL', 'Image', 'Tag', 'Price', 'Price1',
       'style_component__qn84q', 'style_listingdetail__b0bdg',
       'style_listingdetail__b0bdg2', 'style_listingdetail__b0bdg3'],
      dtype='object')

In [106]:
houses_clean = houses.copy()

In [107]:
#keep relevant columns
houses_clean = houses_clean[['Title','Title_URL','Price','Price1','style_listingdetail__b0bdg','style_listingdetail__b0bdg2','style_listingdetail__b0bdg3','style_component__qn84q']]

In [108]:
#rename columns

dict = {'Title':'Street','Title_URL':'Street_URL',
        'Price': 'SoldPrice',
        'Price1':'ListPrice','style_listingdetail__b0bdg':'Bed',
       'style_listingdetail__b0bdg2':'Bath','style_listingdetail__b0bdg3':'Size',
       'style_component__qn84q':'SoldTime'}

houses_clean.rename(columns=dict,
          inplace=True)

In [109]:
#reorder columns
houses_clean = houses_clean[[ 'ListPrice','SoldPrice','SoldTime','Bed','Bath','Size','Street', 'Street_URL']]

In [110]:
houses_clean.head()

,ListPrice,SoldPrice,SoldTime,Bed,Bath,Size,Street,Street_URL
0,"List Price$950,000","$930,000",2 days,3 Bed,2 Bath,1500–2000 sqft,362 Newgate Ave,https://www.zoocasa.com/oshawa-on-real-estate/...
1,"List Price$699,000","$730,000",3 days,3 Bed,2 Bath,N/A sqft,1488 Fenelon Cres,https://www.zoocasa.com/oshawa-on-real-estate/...
2,"List Price$759,300","$715,000",2 days,3 Bed,3 Bath,N/A sqft,536 Park Rd S,https://www.zoocasa.com/oshawa-on-real-estate/...
3,"List Price$599,000","$585,880",Sold 2 Days Ago,2 Bed,2 Bath,N/A sqft,292 Elgin St W,https://www.zoocasa.com/oshawa-on-real-estate/...
4,"List Price$658,000","$698,000",3 days,3 Bed,1 Bath,N/A sqft,148 Colborne St E,https://www.zoocasa.com/oshawa-on-real-estate/...


In [112]:
#strip numeric columns of unwanted characters and convert to numeric
#will deal with size seperately

cols = ['ListPrice','SoldPrice','Bed','Bath']

houses_clean[cols]=houses_clean[cols].apply(lambda x: x.str.replace(r'\D', ''))
houses_clean[cols] = houses_clean[cols].apply(pd.to_numeric, errors='coerce')

In [113]:
houses_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53632 entries, 0 to 10268
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ListPrice   53632 non-null  int64 
 1   SoldPrice   53632 non-null  int64 
 2   SoldTime    53632 non-null  object
 3   Bed         53632 non-null  int64 
 4   Bath        53632 non-null  int64 
 5   Size        53632 non-null  object
 6   Street      53632 non-null  object
 7   Street_URL  53632 non-null  object
dtypes: int64(4), object(4)
memory usage: 3.7+ MB


- Size has a range so I'll take the mean of the two numbers to get a size gives an estimate of the size. 
- I'll fill the NA with the mean of the sizes because that's one way to deal with missing values(another is to remove them but I won't be doing that) 

In [118]:
#split original size column
houses_clean[['A', 'B']] = houses_clean['Size'].str.split('–', 1, expand=True)


In [132]:
#replace non digits, convert to numeric and then get mean of both numbers
sz_cols = ['A','B']

houses_clean[sz_cols]=houses_clean[sz_cols].apply(lambda x: x.str.replace(r'\D', ''))
houses_clean[sz_cols] = houses_clean[sz_cols].apply(pd.to_numeric, errors='coerce')
houses_clean['sqft'] = houses_clean[['A', 'B']].mean(axis=1)

#replace na with mean 
mean = np.mean(houses_clean['sqft'])
houses_clean['sqft']=houses_clean['sqft'].fillna(mean)

In [ ]:
#drop original size column and A, B
houses_clean.drop(columns=['Size','A','B'], axis=1, inplace=True)

Need to find the cities of the house. This is somewhere in the street url after '.com'

In [129]:
#split street url by / and select the last index e.g 'oshawa-on-real-estate'
houses_clean['Street_URL']= houses_clean['Street_URL'].apply(lambda x: x.split('/')[3])

#replace to get just the city and rename col
houses_clean['Street_URL']=houses_clean['Street_URL'].apply(lambda x: x.replace('-on-real-estate',''))
houses_clean.rename(columns={'Street_URL': 'City'}, inplace=True)


In [134]:
houses_clean.head()

,ListPrice,SoldPrice,SoldTime,Bed,Bath,Street,City,sqft
0,950000,930000,2 days,3,2,362 Newgate Ave,oshawa,1750.000000
1,699000,730000,3 days,3,2,1488 Fenelon Cres,oshawa,1811.364667
2,759300,715000,2 days,3,3,536 Park Rd S,oshawa,1811.364667
3,599000,585880,Sold 2 Days Ago,2,2,292 Elgin St W,oshawa,1811.364667
4,658000,698000,3 days,3,1,148 Colborne St E,oshawa,1811.364667


- SoldTime needs to be cleaned 
- During the scraping, I could not get the actual date the houses were sold, so I will do a best estimate using the sold time. I will subtract the sold time from the date I scraped the data 

In [75]:

 pd.to_numeric(houses_clean['Size']
    .str.replace('[^0-9\- ]', '') # 1. remove non-(digit/dash/space) characters
    .str.split('-')             # 2. split by ' - '
    .explode()                    # 3. explode the lists
).mean(level=0)                   # 4. calculate average by index


0        1.220156e+07
1        1.714494e+07
2        1.907419e+07
3        1.592707e+07
4        1.381296e+07
             ...     
11402    1.600180e+07
11403    2.500300e+07
11404             NaN
11405             NaN
11406    1.000120e+07
Name: Size, Length: 11407, dtype: float64

In [85]:
houses_clean['Size'].replace('N/A sqft',0, inplace=True)
houses_clean['Size'].replace(' ','', inplace=True)
#houses_clean['Size'].replace("0-499 ",0, inplace=True)



In [ ]:
houses_clean['City'].value_counts()